In [93]:
import os
import tempfile
import unittest
import filecmp
import sbol3
import paml
import tyto
import uml
import json

import paml_convert.autoprotocol.plate_coordinates as pc
from paml_convert.autoprotocol.autoprotocol_specialization import AutoprotocolSpecialization

from autoprotocol.protocol import \
    Spectrophotometry, \
    Protocol, \
    WellGroup, \
    Unit
from autoprotocol import container_type as ctype
from paml_convert.autoprotocol.transcriptic_api import TranscripticAPI, TranscripticConfig
from paml.execution_engine import ExecutionEngine

from container_api.client_api import matching_containers, strateos_id

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [98]:
## Execute a PAML protocol, emitting an Autoprotocol protocol

# Setup PAML protocol for execution
out_dir = "."
doc = sbol3.Document()
sbol3.set_namespace('https://bbn.com/scratch/')
protocol_file = os.path.join(os.getcwd(), "../test/testfiles", "igem_ludox_test.nt")
doc.read(protocol_file, 'nt')
protocol = doc.find("https://bbn.com/scratch/iGEM_LUDOX_OD_calibration_2018")
agent = sbol3.Agent("test_agent")


# Setup Autoprotocol specialization
out_dir = "out"
if not os.path.exists(out_dir):
    os.mkdir(out_dir)
autoprotocol_output = os.path.join(out_dir, "test_LUDOX_autoprotocol.json")
secrets_file = os.path.join(os.getcwd(), "../secrets/tx_secrets.json")
resolutions = {
    doc.find("https://bbn.com/scratch/LUDOX") : "rs1b6z2vgatkq7",
    doc.find("https://bbn.com/scratch/ddH2O") : "rs1c7pg8qs22dt"
}
api = TranscripticAPI(cfg=TranscripticConfig.from_file(secrets_file))
autoprotocol_specialization = AutoprotocolSpecialization(autoprotocol_output, api, resolutions)


# Execute the protocol
ee = ExecutionEngine(specializations=[autoprotocol_specialization])
parameter_values = [
    paml.ParameterValue(parameter=protocol.get_input("wavelength"), 
                        value=uml.LiteralIdentified(value=sbol3.Measure(100, tyto.OM.nanometer)))
]
execution = ee.execute(protocol, agent, id="test_execution", parameter_values=parameter_values)


ee.specializations[0].protocol.instructions

[Instruction(provision, {'resource_id': 'rs1c7pg8qs22dt', 'measurement_mode': 'volume', 'to': [{'well': Well(Container(var_0), 0, 100:microliter), 'volume': Unit(100.0, 'microliter')}, {'well': Well(Container(var_0), 12, 100:microliter), 'volume': Unit(100.0, 'microliter')}, {'well': Well(Container(var_0), 24, 100:microliter), 'volume': Unit(100.0, 'microliter')}, {'well': Well(Container(var_0), 36, 100:microliter), 'volume': Unit(100.0, 'microliter')}]}, []),
 Instruction(provision, {'resource_id': 'rs1b6z2vgatkq7', 'measurement_mode': 'volume', 'to': [{'well': Well(Container(var_0), 1, 100:microliter), 'volume': Unit(100.0, 'microliter')}, {'well': Well(Container(var_0), 13, 100:microliter), 'volume': Unit(100.0, 'microliter')}, {'well': Well(Container(var_0), 25, 100:microliter), 'volume': Unit(100.0, 'microliter')}, {'well': Well(Container(var_0), 37, 100:microliter), 'volume': Unit(100.0, 'microliter')}]}, []),
 Instruction(spectrophotometry, {'dataref': 'measure_absorbance', 'obj

In [100]:
# Submit Autoprotocol object for execution at Strateos

tx = api.get_transcriptic_connection()
response = tx.analyze_run(ee.specializations[0].protocol, test_mode=True)
#response = tx.submit_run(ee.specializations[0].protocol, 
#                         project_id=api.cfg.project_id,
#                         test_mode=True)

response

PermissionError: [403] You are not authorized to execute this command. For more information on access permissions see the package documentation.

In [53]:
from autoprotocol import container_type as ctype
{x: getattr(ctype, x) for x in dir(ctype)}

{'ContainerType': autoprotocol.container_type.ContainerType,
 'DEEP24': ContainerType(name='24-well extended capacity plate', is_tube=False, well_count=24, well_depth_mm=None, well_volume_ul=Unit(10000.0, 'microliter'), well_coating=None, sterile=False, cover_types=None, seal_types=['foil', 'breathable'], capabilities=['liquid_handle', 'incubate', 'gel_separate', 'gel_purify', 'dispense', 'seal'], shortname='24-deep', col_count=6, dead_volume_ul=Unit(15, 'microliter'), safe_min_volume_ul=Unit(60, 'microliter'), true_max_vol_ul=Unit(10000.0, 'microliter'), vendor='E&K Scientific', cat_no='EK-2053-S', prioritize_seal_or_cover='seal'),
 'DEEP96': ContainerType(name='96-well extended capacity plate', is_tube=False, well_count=96, well_depth_mm=None, well_volume_ul=Unit(2000.0, 'microliter'), well_coating=None, sterile=False, cover_types=['standard', 'universal'], seal_types=['breathable'], capabilities=['liquid_handle', 'incubate', 'gel_separate', 'gel_purify', 'cover', 'dispense', 'seal']

In [45]:
spec = doc.find("https://bbn.com/scratch/iGEM_LUDOX_OD_calibration_2018/CallBehaviorAction1/ValuePin1/LiteralIdentified1/ContainerSpec1")

print(f"queryString: {spec.queryString}")
print(f"prefixMap: {spec.prefixMap}")
matches = matching_containers(spec)
matches

queryString: cont:ClearPlate and 
 cont:SLAS-4-2004 and
 (cont:wellVolume some 
    ((om:hasUnit value om:microlitre) and
     (om:hasNumericalValue only xsd:decimal[>= "200"^^xsd:decimal])))
prefixMap: {"cont": "https://sift.net/container-ontology/container-ontology#", "om": "http://www.ontology-of-units-of-measure.org/resource/om-2/"}


['https://sift.net/container-ontology/strateos-catalog.ttl#Eppendorf96-flat-clear-clear-tc',
 'https://sift.net/container-ontology/bbn-plate-catalog#Corning_96_well_black_clear_bottom_plates',
 'https://sift.net/container-ontology/bbn-plate-catalog#NEST_0.1_mL_96-Well_PCR_Plate_Full_Skirt',
 'https://sift.net/container-ontology/strateos-catalog.ttl#Costar96-flat-clear-costar-3590',
 'https://sift.net/container-ontology/strateos-catalog.ttl#Corning96-ubottom-clear-tc']

In [46]:
for x in matches:
    try:
        c = strateos_id(x) 
        print(f"Searching inventory for [{c}]")
        result = tx.inventory(c)
        print(result)
    except Exception as e:
        continue

Searching inventory for [96-flat-clear-clear-tc]
{'results': [], 'num_pages': 0, 'per_page': 75}
Searching inventory for [_96_well_black_clear_bottom_plates]
{'results': [], 'num_pages': 0, 'per_page': 75}
Searching inventory for [96-flat-clear-costar-3590]
{'results': [], 'num_pages': 0, 'per_page': 75}
Searching inventory for [96-ubottom-clear-tc]
{'results': [], 'num_pages': 0, 'per_page': 75}


In [90]:
import transcriptic
c = transcriptic.Container("ct1g9prj2sz44xc")
c.container_type.

ContainerType(name='96-well flat-bottom UV transparent plate', is_tube=False, well_count=96, well_depth_mm=None, well_volume_ul=Unit(340.0, 'microliter'), well_coating=None, sterile=False, cover_types=['low_evaporation', 'standard', 'universal'], seal_types=None, capabilities=['liquid_handle', 'spin', 'absorbance', 'fluorescence', 'luminescence', 'incubate', 'gel_separate', 'gel_purify', 'cover', 'dispense'], shortname='96-flat-uv', col_count=12, dead_volume_ul=Unit(25, 'microliter'), safe_min_volume_ul=Unit(65, 'microliter'), true_max_vol_ul=Unit(340.0, 'microliter'), vendor='Corning', cat_no='3635', prioritize_seal_or_cover='seal')

# The following is Scratch space

In [76]:
tx.inventory("flat test")['results'][0]
#tx_protocols = tx.get_protocols()

{'id': 'ct1arq6n82p9xu',
 'container_type_id': '96-pcr',
 'barcode': None,
 'deleted_at': None,
 'created_at': '2017-10-05T15:06:57.960-07:00',
 'organization_id': 'org1amxh23ednpz',
 'slot': None,
 'cover': 'ultra-clear',
 'test_mode': True,
 'label': 'test 96-flat',
 'location_id': None,
 'shipment_id': None,
 'kit_request_id': None,
 'storage_condition': 'cold_4',
 'shipment_code': None,
 'status': 'inbound',
 'expires_at': None,
 'created_by': None,
 'aliquot_count': 96,
 'container_type': {'id': '96-pcr',
  'name': '96-Well twin.tec PCR Plate',
  'well_count': 96,
  'well_depth_mm': '14.6',
  'well_volume_ul': '160.0',
  'capabilities': ['bluewash',
   'dispense-destination',
   'echo_dest',
   'envision',
   'flash_freeze',
   'image_plate',
   'incubate',
   'magnetic_separate',
   'magnetic_transfer_pcr',
   'miniprep_destination',
   'qrt_thermocycle',
   'sanger_sequence',
   'seal',
   'spin',
   'stamp',
   'thermocycle'],
  'shortname': '96-pcr',
  'col_count': 12,
  'is_t

In [96]:
api._protocol_name_map["MakeContainers"].protocol

{'id': 'pr1g9ptfcgt2wab',
 'name': 'MakeContainers',
 'created_at': '2021-10-12T13:28:36.333-07:00',
 'description': 'Module that allows creation of containers, with names and attributes.',
 'package_name': 'com.sd2org.ObstacleCourse',
 'package_id': 'pk1dy3v7twx3bne',
 'release_id': 're1g9ptf8h4jc52',
 'inputs': {'containers': {'type': 'group+',
   'label': 'Container',
   'description': 'Metadata for container creation',
   'inputs': {'name': {'type': 'string',
     'label': 'Container Name',
     'description': 'Name of container. Containers in the same run cannot share the same name.',
     'default': 'Name'},
    'cont_type': {'type': 'choice',
     'label': 'Container Type',
     'default': 'micro-1.5',
     'options': [{'value': 'micro-1.5',
       'name': 'Microcentrifuge Tube - 1.5mL'},
      {'value': 'micro-2.0', 'name': 'Microcentrifuge Tube - 2.0mL'},
      {'value': 'conical-15', 'name': '15mL Conical'},
      {'value': '96-flat', 'name': '96-flat'},
      {'value': '96-p

In [97]:
 container_spec = {
                "name": "test create PAML container",
                "cont_type": ctype.FLAT96.shortname, # resolve with spec here
                "volume": "100:microliter", # FIXME where does this come from?
                "properties": [
                    {
                        "key": "concentration",
                        "value": "10:millimolar"
                    }
                ]
            }
api.make_containers([container_spec])

Launching: launch_request_id = lr1g9q3agtmucfs
Launching: protocol_id = pr1g9ptfcgt2wab
Launching: project_id = p1bqm3ehqzgum
Launching: title = 2021-10-12T22:15:25T871754_make_containers_MakeContainers
Launching: test_mode = True


{'id': 'r1g9q3bndcsr4q',
 'status': 'accepted',
 'title': '2021-10-12T22:15:25T871754_make_containers_MakeContainers',
 'created_at': '2021-10-12T15:15:25.947-07:00',
 'updated_at': '2021-10-12T15:15:26.000-07:00',
 'completed_at': None,
 'conversation_id': 'conv1g9q3bne4gpqc',
 'quote': {'items': [{'cost': '0.69',
    'quantity': 1,
    'title': 'Workcell Time',
    'run_id': 'r1g9q3bndcsr4q',
    'run_credit_applicable': True},
   {'cost': '11.87',
    'quantity': 1,
    'title': 'Reagents & Consumables',
    'run_id': 'r1g9q3bndcsr4q',
    'run_credit_applicable': False}],
  'ppp': False},
 'results': {},
 'test_mode': True,
 'accepted_by_id': 'u1b64bbpxftks',
 'accepted_at': '2021-10-12T15:15:25.947-07:00',
 'started_at': None,
 'canceled_at': None,
 'aborted_at': None,
 'predecessor_id': None,
 'draft_quote': None,
 'progress': 0,
 'protocol_id': 'pr1g9ptfcgt2wab',
 'request_type': 'protocol',
 'launch_request_id': 'lr1g9q3agtmucfs',
 'flagged': False,
 'estimated_run_time_cache':

In [103]:
tx.inventory("96-flat")

{'results': [{'id': 'ct1arq6n82p9xu',
   'container_type_id': '96-pcr',
   'barcode': None,
   'deleted_at': None,
   'created_at': '2017-10-05T15:06:57.960-07:00',
   'organization_id': 'org1amxh23ednpz',
   'slot': None,
   'cover': 'ultra-clear',
   'test_mode': True,
   'label': 'test 96-flat',
   'location_id': None,
   'shipment_id': None,
   'kit_request_id': None,
   'storage_condition': 'cold_4',
   'shipment_code': None,
   'status': 'inbound',
   'expires_at': None,
   'created_by': None,
   'aliquot_count': 96,
   'container_type': {'id': '96-pcr',
    'name': '96-Well twin.tec PCR Plate',
    'well_count': 96,
    'well_depth_mm': '14.6',
    'well_volume_ul': '160.0',
    'capabilities': ['bluewash',
     'dispense-destination',
     'echo_dest',
     'envision',
     'flash_freeze',
     'image_plate',
     'incubate',
     'magnetic_separate',
     'magnetic_transfer_pcr',
     'miniprep_destination',
     'qrt_thermocycle',
     'sanger_sequence',
     'seal',
     'sp

In [ ]:
#[n.identity for n in protocol.nodes if isinstance(n, uml.CallBehaviorAction)]
provision = doc.find("https://bbn.com/scratch/iGEM_LUDOX_OD_calibration_2018/CallBehaviorAction3")
parameters = [x.property_value for x in provision.behavior.lookup().parameters]
[(x., x.name) for x in parameters]

In [ ]:
import requests
resource = doc.find("https://bbn.com/scratch/LUDOX")
types = resource.types
resolved_types = [requests.get(t).headers['content-type'] for t in types]
resolved_types

In [ ]:
requests.request("https://identifiers.org/pubchem.substance:24866361")

In [ ]:
print(resource.type)
api.resolve_resource(resource)

In [ ]:
tx = api.get_transcriptic_connection()
[result] = tx.resources("ddh2o")["results"]
water_id = result["id"]
[result] = tx.resources("ludox")["results"]
ludox_id = result["id"]

resolutions = {specialization.unresolved_terms[0] : 'abc',
               specialization.unresolved_terms[1] : water_id,
               specialization.unresolved_terms[2] : ludox_id}
resolved_terms = [ResolvedTerm(x, resolutions[x]) for x in specialization.unresolved_terms]
resolved_terms

In [ ]:
print([x.unresolved_term.step for x in resolved_terms])
specialization.resolve(resolved_terms)


In [ ]:
import pubchempy
import pandas as pd
substance = "24901740"
s = pubchempy.Substance.from_sid(substance)


results = api.get_transcriptic_connection().resources(s.synonyms[0].split(" ")[0])
{r['id'] : (r['name'], r['pub_chem_id']) for r in results['results'] if 'pub_chem_id' in r}
#{r['id'] : list(r.keys()) for r in results['results']}
set([k for r in results['results'] for k in r.keys()])

results
#df = pd.DataFrame({r['id'] : [list(r.keys())] for r in results['results']})
#df

#import requests
#r = requests.get("https://identifiers.org/pubchem.substance:24901740")
#r.content